In [ ]:
!git clone https://github.com/zheliu17/nanoRecSys.git
%pip install -q -e ./nanoRecSys


# In fact, we don't need psutil. force-reinstall to trigger colab restart
%pip install --force-reinstall psutil=={psutil.__version__}
print("Installation complete. Please restart runtime...")

In [ ]:
import os

os.environ["retrieval_threshold".upper()] = "0"


import nanoRecSys.data

nanoRecSys.data.build_dataset.process_data()
# Leave-one-out
nanoRecSys.data.splits.create_user_time_split(1, 1)

In [ ]:
import nanoRecSys.train


class Args:
    mode = "retriever"
    epochs = 5
    batch_size = 4096
    lr = 4e-2
    num_workers = 2
    build_embeddings = True


nanoRecSys.train.main(Args)

These metrics are comparable to the paper values. For the popularity baseline, numbers below are similar to
https://arxiv.org/abs/1904.06690

Note: MRR in paper is global MRR, and should be compared to MRR@100 here.

In [ ]:
from nanoRecSys.eval.offline_eval import OfflineEvaluator

evaluator = OfflineEvaluator(1024, sampled=True, sample_strategy="popularity")
results = evaluator.eval_popularity()

df = evaluator.formatted_results(results)
df

Not surprisingly, our model beats NCF ([Neural collaborative filtering](https://dl.acm.org/doi/abs/10.1145/3038912.3052569)).

But lower than the true sequential retrievers.

See numbers in https://arxiv.org/abs/1904.06690

In [ ]:
results = evaluator.eval_retrieval()
df = evaluator.formatted_results(results)
df